# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
##Checking the current working directory
print(os.getcwd())

##Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

##Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
##Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
##Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
##For every filepath in the file path list 
for f in file_path_list:

##Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 ##Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

##Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
##Check the number of rows in the newly created csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The Apache Cassandra Part. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above has been run successfully:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
##This should make a connection to a Cassandra instance with local machine 
## (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()
# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
##Creating Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
                    WITH REPLICATION = 
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                   )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
##Setting the KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


## While creating data model for a NoSQL database, we should always build our data model with keeping in mind the queries we need to run.

## Here are our required queries:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## 1- The First Query:

In [8]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Rokia TraorÃÂ©,Stefany,F,0,White,274.88608,free,"Lubbock, TX",693,Zen,83
1,Camila,Tucker,M,1,Garrison,230.81751,free,"Oxnard-Thousand Oaks-Ventura, CA",555,Abrazame (Version Acustica),40
2,Carl Thomas,Tucker,M,0,Garrison,196.67546,free,"Oxnard-Thousand Oaks-Ventura, CA",698,You Ain't Right (Album Version),40
3,N.E.R.D.,James,M,0,Martin,242.99057,free,"Dallas-Fort Worth-Arlington, TX",78,Provider (Remix Radio Edit),79
4,Lil Jon / The East Side Boyz / DJ Flexx,Jacqueline,F,3,Lynch,285.30893,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Aww Skeet Skeet,29


In [9]:
query1_df = df.query("sessionId == '338'")
query1_df

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5006,Pixies,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",338,Build High,50
5007,The Roots / Jack Davey,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",338,Atonement,50
5008,Mike And The Mechanics,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",338,A Beggar On A Beach Of Gold,50
5009,Faithless,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [11]:
##Creating first table to answer first query:
query = "CREATE TABLE IF NOT EXISTS SONG_INFO "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### In case of re-processing, DROP TABLE Statement has been added.

In [10]:
##droping first table in case of re-processing
query = "DROP TABLE IF EXISTS SONG_INFO"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
##The following code should read all lines from the newly created CSV file and extract specific column's values
##and insert these values in table 'SONG_INFO'
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ##Assigning here the INSERT statements into the `query` variable
        query = "INSERT INTO SONG_INFO (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ##Selecting here which element of each line (needed Columns) we will insert in the Table 'SONG_INFO'
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### A verification step:

This query is designed specifically to get the name of the artist, the song title and the song length from table SONG_INFO. While creating the mentioned table, i thought creating a PRIMARY KEY consists of partition key (session_id) and clustring column (item_in_session), would help this table to achieve low latency measure in this NoSQL DB.

Thus, I implemented the above idea regarding PRIMARY KEY, defining the PRIMARY KEY as combination of 1 PARTITION KEY 'sessionId' and 1 CLUSTRING COLUMN 'itemInSession'. This implementation will lead us to have the expected results (4 rows), not a random row in case i choose to create the table without specific PRIMARY KEY.

In [13]:
##Verifying that the table 'SONG_INFO' has been filled properly
query = "SELECT artist, song_title, song_length FROM SONG_INFO WHERE session_id = 338 and item_in_session  = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## 2- The Second Query:

In [14]:
query2_df = df.query('sessionId == "182" and userId == "10"')
query2_df

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1918,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
1919,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
1920,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
1921,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [17]:
##Creating seond table to answer second query:
query = "CREATE TABLE IF NOT EXISTS SESSION_USER_INFO"
query = query + "(session_id int, user_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY (session_id, user_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### In case of re-processing, DROP TABLE Statement has been added.

In [16]:
##droping second table in case of re-processing
query = "DROP TABLE IF EXISTS SESSION_USER_INFO"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
##The following code should read all lines from the newly created CSV file and extract specific column's values
##and insert these values in table 'SESSION_USER_INFO'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ##Assigning here the INSERT statements into the `query` variable
        query = "INSERT INTO SESSION_USER_INFO (session_id, user_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ##Selecting here which element of each line (needed Columns) we will insert in the Table 'SESSION_USER_INFO'
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[1], line[4], line[9]))

#### A verification step:

This query is designed specifically to get the name of the artist, the song title and the song length from table SESSION_USER_INFO. While creating the mentioned table, i thought creating a PRIMARY KEY consists of partition key (session_id) and clustring columns (user_id, item_in_session), would help this table to achieve low latency measure in this NoSQL DB.

Thus, I implemented the above idea regarding PRIMARY KEY, defining the PRIMARY KEY as combination of 1 PARTITION KEY 'sessionId' and 2 CLUSTRING COLUMN 'userId' and 'itemInSession'. This implementation will lead us to have the expected results (4 rows), not a random row in case i choose to create the table without specific PRIMARY KEY.

In [19]:
##Verifying that the table 'SESSION_USER_INFO' has been filled properly
query = "SELECT artist, song_title, first_name, last_name FROM SESSION_USER_INFO WHERE SESSION_ID=182 AND USER_ID=10;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


## 3- The Third Query:

In [20]:
query3_df = df.query('song == "All Hands Against His Own"')
query3_df

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
222,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
1658,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
2797,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


In [22]:
##Creating Third table to answer third query:
query = "CREATE TABLE IF NOT EXISTS USER_INFO"
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### In case of re-processing, DROP TABLE Statement has been added.

In [21]:
##droping second table in case of re-processing
query = "DROP TABLE IF EXISTS USER_INFO"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [23]:
##The following code should read all lines from the newly created CSV file and extract specific column's values
##and insert these values in table 'USER_INFO'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ##Assigning here the INSERT statements into the `query` variable
        query = "INSERT INTO USER_INFO (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ##Selecting here which element of each line (needed Columns) we will insert in the Table 'USER_INFO'
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### A verification step:

This query is designed specifically to get the name of the artist, the song title and the song length from table USER_INFO. While creating the mentioned table, i thought creating a PRIMARY KEY consists of partition key (song_title) and clustring columns (user_id), would help this table to achieve low latency measure in this NoSQL DB.

Thus, I implemented the above idea regarding PRIMARY KEY, defining the PRIMARY KEY as combination of 1 PARTITION KEY 'sessionId' and 1 CLUSTRING COLUMN 'userId'. This implementation will lead us to have the expected results (3 rows), not a random row in case i choose to create the table without specific PRIMARY KEY.

In [24]:
##Verifying that the table 'USER_INFO' has been filled properly
query = "SELECT first_name, last_name FROM USER_INFO WHERE SONG_TITLE = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [25]:
##Dropping the 'SONG_INFO' before closing out the session
query = "DROP TABLE IF EXISTS SONG_INFO"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [26]:
##Dropping the 'SESSION_USER_INFO' before closing out the session
query = "DROP TABLE IF EXISTS SESSION_USER_INFO"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [27]:
##Dropping the 'SESSION_USER_INFO' before closing out the session
query = "DROP TABLE IF EXISTS USER_INFO"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()